In [ ]:
!pip install gradio transformers torch accelerate bitsandbytes

import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import gc

# Model listesi - Hem Toprak Henaz hem de ByGedik modelleri
MODELS = {
    # Toprak Henaz Modelleri
    "DeepSeek Coder 6.7B (ToprakH)": "toprakhenaz/DeepSeek-Coder-6.7B-Instruct",
    "CodeLlama Magicoder 7B (ToprakH)": "toprakhenaz/codellama_magicoder-7b-lora-adapters",
    "Gemma 7B (ToprakH)": "toprakhenaz/gemma-7b-lora-adapters",
    "Qwen2.5 Coder 7B (ToprakH)": "toprakhenaz/qwen2.5-coder-7b-lora-adapters",

    # ByGedik Modelleri
    "Llama 3.1 8B CodeAlpaca (ByGedik)": "ByGedik/Meta-Llama-3.1-8B-Instruct-bnb-4bit-codeAlpaca-CodePlusExplanation",
    "Mistral CodeAlpaca V3 (ByGedik)": "ByGedik/mistral-codealpaca-v3epoch"
}

# Global değişkenler
current_model = None
current_tokenizer = None
current_model_name = None

# Model cache sistemi
model_cache = {}  # {model_name: {"model": model, "tokenizer": tokenizer, "last_used": timestamp}}
max_cached_models = 3  # Maksimum cache'lenecek model sayısı
import time

# 4-bit quantization config (GPU bellek kullanımını azaltmak için)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

def load_model(model_name):
    """Seçilen modeli yükle veya cache'den al"""
    global current_model, current_tokenizer, current_model_name, model_cache

    if current_model_name == model_name:
        return f"✅ {model_name} zaten aktif!"

    # Cache'de var mı kontrol et
    if model_name in model_cache:
        # Cache'den al
        current_model = model_cache[model_name]["model"]
        current_tokenizer = model_cache[model_name]["tokenizer"]
        current_model_name = model_name
        model_cache[model_name]["last_used"] = time.time()

        cache_info = get_cache_info()
        return f"🚀 {model_name} cache'den hızlıca yüklendi!\n{cache_info}"

    try:
        model_path = MODELS[model_name]

        # Cache boyut kontrolü - gerekirse eski modeli sil
        if len(model_cache) >= max_cached_models:
            remove_oldest_from_cache()

        # Tokenizer'ı yükle
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        # Modeli yükle
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            quantization_config=quantization_config,
            device_map="auto",
            torch_dtype=torch.float16,
            trust_remote_code=True
        )

        # Cache'e ekle
        model_cache[model_name] = {
            "model": model,
            "tokenizer": tokenizer,
            "last_used": time.time()
        }

        # Aktif model olarak ayarla
        current_model = model
        current_tokenizer = tokenizer
        current_model_name = model_name

        cache_info = get_cache_info()
        return f"✅ {model_name} başarıyla yüklendi ve cache'lendi!\n{cache_info}"

    except Exception as e:
        return f"❌ Model yüklenirken hata: {str(e)}"

def remove_oldest_from_cache():
    """En eski kullanılan modeli cache'den sil"""
    global model_cache

    if not model_cache:
        return

    # En eski modeli bul
    oldest_model = min(model_cache.keys(),
                      key=lambda x: model_cache[x]["last_used"])

    # Cache'den sil
    del model_cache[oldest_model]["model"]
    del model_cache[oldest_model]["tokenizer"]
    del model_cache[oldest_model]

    # GPU belleğini temizle
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def get_cache_info():
    """Cache durumunu göster"""
    global model_cache

    if not model_cache:
        return "📦 Cache boş"

    cache_list = []
    for name in model_cache.keys():
        last_used = model_cache[name]["last_used"]
        time_diff = int(time.time() - last_used)
        cache_list.append(f"• {name} ({time_diff}s önce)")

    return f"📦 Cache'deki modeller ({len(model_cache)}/{max_cached_models}):\n" + "\n".join(cache_list)

def generate_response(prompt, max_length=512, temperature=0.7, top_p=0.9):
    """Seçili model ile metin üret"""
    global current_model, current_tokenizer

    if current_model is None:
        return "❌ Lütfen önce bir model seçin ve yükleyin!"

    try:
        # Prompt'u encode et
        inputs = current_tokenizer.encode(prompt, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = inputs.cuda()

        # Metin üret
        with torch.no_grad():
            outputs = current_model.generate(
                inputs,
                max_length=max_length,
                temperature=temperature,
                top_p=top_p,
                do_sample=True,
                pad_token_id=current_tokenizer.eos_token_id,
                eos_token_id=current_tokenizer.eos_token_id
            )

        # Decode et ve prompt'u çıkar
        generated_text = current_tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = generated_text[len(prompt):].strip()

        return response

    except Exception as e:
        return f"❌ Metin üretilirken hata: {str(e)}"

def clear_memory():
    """Tüm cache'i ve belleği temizle"""
    global current_model, current_tokenizer, current_model_name, model_cache

    # Tüm cache'lenmiş modelleri temizle
    for cached_model in model_cache.values():
        del cached_model["model"]
        del cached_model["tokenizer"]

    model_cache.clear()
    current_model = None
    current_tokenizer = None
    current_model_name = None

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return "🧹 Tüm cache ve bellek temizlendi!"

def clear_single_model(model_name):
    """Tek bir modeli cache'den sil"""
    global model_cache, current_model, current_tokenizer, current_model_name

    if model_name not in model_cache:
        return f"❌ {model_name} cache'de bulunamadı!"

    # Aktif model ise sıfırla
    if current_model_name == model_name:
        current_model = None
        current_tokenizer = None
        current_model_name = None

    # Cache'den sil
    del model_cache[model_name]["model"]
    del model_cache[model_name]["tokenizer"]
    del model_cache[model_name]

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    cache_info = get_cache_info()
    return f"🗑️ {model_name} cache'den silindi!\n{cache_info}"

# Gradio arayüzü
def create_interface():
    with gr.Blocks(title="Model Switcher", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🤖 Multi-Profile Model Switcher")
        gr.Markdown("Toprak Henaz ve ByGedik profillerindeki modelleri seçin ve kullanın!")

        with gr.Row():
            with gr.Column(scale=1):
                model_dropdown = gr.Dropdown(
                    choices=list(MODELS.keys()),
                    label="🎯 Model Seçin",
                    value=list(MODELS.keys())[0]
                )

                with gr.Row():
                    load_btn = gr.Button("📥 Model Yükle", variant="primary")
                    clear_btn = gr.Button("🧹 Tümünü Temizle", variant="secondary")

                # Cache yönetimi
                with gr.Row():
                    cache_info_btn = gr.Button("📦 Cache Durumu", variant="secondary")
                    clear_single_btn = gr.Button("🗑️ Bu Modeli Sil", variant="secondary")

                status_text = gr.Textbox(
                    label="📊 Durum & Cache Bilgisi",
                    value="Model seçin ve yükleyin...",
                    interactive=False,
                    lines=4
                )

            with gr.Column(scale=2):
                prompt_input = gr.Textbox(
                    label="✍️ Prompt Girin",
                    placeholder="Kodlama sorunuz veya metin prompt'ınızı buraya yazın...",
                    lines=3
                )

                with gr.Row():
                    max_length_slider = gr.Slider(
                        minimum=50,
                        maximum=1024,
                        value=512,
                        step=50,
                        label="📏 Maksimum Uzunluk"
                    )

                    temperature_slider = gr.Slider(
                        minimum=0.1,
                        maximum=2.0,
                        value=0.7,
                        step=0.1,
                        label="🌡️ Temperature"
                    )

                    top_p_slider = gr.Slider(
                        minimum=0.1,
                        maximum=1.0,
                        value=0.9,
                        step=0.1,
                        label="🎯 Top-p"
                    )

                generate_btn = gr.Button("🚀 Üret", variant="primary", size="lg")

                output_text = gr.Textbox(
                    label="📝 Çıktı",
                    lines=10,
                    interactive=False
                )

        # Event handlers
        load_btn.click(
            fn=load_model,
            inputs=[model_dropdown],
            outputs=[status_text]
        )

        clear_btn.click(
            fn=clear_memory,
            outputs=[status_text]
        )

        cache_info_btn.click(
            fn=get_cache_info,
            outputs=[status_text]
        )

        clear_single_btn.click(
            fn=clear_single_model,
            inputs=[model_dropdown],
            outputs=[status_text]
        )

        generate_btn.click(
            fn=generate_response,
            inputs=[prompt_input, max_length_slider, temperature_slider, top_p_slider],
            outputs=[output_text]
        )

        # Örnekler
        gr.Examples(
            examples=[
                ["Python'da bir fibonacci fonksiyonu yaz ve açıkla"],
                ["JavaScript ile REST API çağrısı nasıl yapılır?"],
                ["SQL ile JOIN işlemleri örneği"],
                ["React functional component ile state management"],
                ["Machine Learning model evaluation metrikleri"],
                ["Docker container oluşturma adımları"],
            ],
            inputs=[prompt_input]
        )

        gr.Markdown("""
        ### 🎯 **Mevcut Modeller (6 adet)**:

        **Toprak Henaz Profili:**
        - DeepSeek Coder 6.7B (Kod üretimi)
        - CodeLlama Magicoder 7B (Kod tamamlama)
        - Gemma 7B (Genel amaçlı)
        - Qwen2.5 Coder 7B (Kod analizi)

        **ByGedik Profili:**
        - Llama 3.1 8B CodeAlpaca (Kod açıklaması)
        - Mistral CodeAlpaca V3 (Kod örnekleri)

        ### 📋 Kullanım Talimatları:
        1. **Model Seçin**: Dropdown'dan kullanmak istediğiniz modeli seçin
        2. **Model Yükle**: "Model Yükle" butonuna tıklayarak modeli cache'e yükleyin
        3. **Hızlı Geçiş**: Cache'lenmiş modeller arası anında geçiş yapın
        4. **Prompt Girin**: Kodlama sorunuzu veya metin prompt'ınızı yazın
        5. **Parametreleri Ayarlayın**: Uzunluk, temperature ve top-p değerlerini ayarlayın
        6. **Üret**: "Üret" butonuna tıklayarak cevap alın

        ### 🚀 Cache Sistemi:
        - **Maksimum 3 model** aynı anda cache'de tutulur
        - **LRU (Least Recently Used)** algoritması ile eski modeller otomatik temizlenir
        - **Anında geçiş** cache'lenmiş modeller arasında
        - **Bellek yönetimi** otomatik optimizasyon

        ### ⚙️ Parametre Açıklamaları:
        - **Maksimum Uzunluk**: Üretilecek metnin maksimum token sayısı
        - **Temperature**: Yaratıcılık seviyesi (düşük = daha tutarlı, yüksek = daha yaratıcı)
        - **Top-p**: Kelime seçimi çeşitliliği (0.9 önerilir)

        ### 💡 İpuçları:
        - **Cache sistemi** sayesinde modeller arası hızlı geçiş yapabilirsiniz
        - **3 model** aynı anda cache'de tutulur, daha fazlası için otomatik temizlik
        - **GPU bellek optimizasyonu** 4-bit quantization ile sağlanır
        - **Cache durumunu** "Cache Durumu" butonu ile kontrol edebilirsiniz
        - **ToprakH modelleri** genel kod üretimi için optimize edilmiş
        - **ByGedik modelleri** CodeAlpaca dataset'i ile fine-tune edilmiş
        - **Model karşılaştırması** için aynı prompt'u farklı modellerde test edin
        - **Bellek temizliği** gerektiğinde "Tümünü Temizle" veya "Bu Modeli Sil" kullanın
        """)

    return demo

# Uygulamayı başlat
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(
        share=True,  # Public link oluştur
        debug=True,
        height=800
    )